In [84]:
def UWPGMA(frame, weighted=True):
    
    import pandas as pd
    import numpy as np
    
    def min_ind(frame):
        ind=frame.values.argmin()
        row=ind//frame.shape[0]
        col=ind%frame.shape[1]
        name_t=(frame.index[row], frame.columns[col])
        return name_t
    
    Weights={key: 1 for key in frame.index}
    
    def weight(el):
        return 1 if weighted else Weights[el]
        
    def weights_upd(el_1, el_2):
        Weights[el_1+el_2]=Weights[el_1]+Weights[el_2]
        
    def format_matrix(frame, el_1, el_2):
        temp=frame.iloc[(frame.index!=el_1) & (frame.index!=el_2), (frame.columns!=el_1) & (frame.columns!=el_2)]
        distances=[]
        
        for c in temp.index:
            f_1=(frame.loc[c, el_1] if frame.loc[c, el_1]!=np.inf else frame.loc[el_1, c])
            f_2=(frame.loc[c, el_2] if frame.loc[c, el_2]!=np.inf else frame.loc[el_2, c])
            dist=(weight(m_name[0])*f_1+weight(m_name[1])*f_2)/(weight(m_name[0])+weight(m_name[1]))
            distances.append(dist)
        
        temp[el_1+el_2]=distances
        row_to_app=pd.Series([np.inf]*temp.shape[1], index=temp.columns, name=el_1+el_2)
        temp=temp.append(row_to_app)
        
        return temp
        
    Tree_struct={}
    
    def merge(el_1, el_2):
        Dist=(frame.loc[el_1, el_2] if frame.loc[el_1, el_2]!=np.inf else frame.loc[el_2, el_1])
        m_1=(Tree_struct[el_1] if el_1 in Tree_struct else el_1)
        m_2=(Tree_struct[el_2] if el_2 in Tree_struct else el_2)
        Tree_struct[el_1+el_2]='({}: {}, {}: {})'.format(m_1, Dist/2, 
                                                         m_2, Dist/2)
            
    while np.sum(frame.values.flatten()!=np.inf)!=1:
        
        m_name=min_ind(frame)
        
        merge(*m_name)
        weights_upd(*m_name)
        frame=format_matrix(frame, *m_name)
        
    final_name=min_ind(frame)
    merge(*final_name)
        
    return frame, Tree_struct[final_name[0]+final_name[1]]

In [5]:
import pandas as pd
import numpy as np

In [48]:
Test_1=pd.DataFrame([[np.inf, 16, 16, 10],[np.inf, np.inf, 8, 8], 
                     [np.inf, np.inf, np.inf, 4], [np.inf, np.inf, np.inf, np.inf]],
                   index=['A', 'B', 'C', 'D'], columns=['A', 'B', 'C', 'D'])
Test_1

,A,B,C,D
A,inf,16.000000,16.000000,10.000000
B,inf,inf,8.000000,8.000000
C,inf,inf,inf,4.000000
D,inf,inf,inf,inf


In [86]:
F, T_s=UWPGMA(Test_1)
F

,A,BCD
A,inf,14.500000
BCD,inf,inf


In [82]:
T_s

'(A: 7.25, (B: 4.0, (C: 2.0, D: 2.0): 4.0): 7.25)'

In [89]:
F, T_s=UWPGMA(Test_1, weighted=False)
F

,A,BCD
A,inf,14.000000
BCD,inf,inf


In [90]:
T_s

'(A: 7.0, (B: 4.0, (C: 2.0, D: 2.0): 4.0): 7.0)'

In [6]:
Test_2=pd.DataFrame([[np.inf, 5, 4, 7, 6, 8],[np.inf, np.inf, 7, 10, 9, 11], 
                     [np.inf, np.inf, np.inf, 7, 6, 8], [np.inf, np.inf, np.inf, np.inf, 5, 9],
                     [np.inf, np.inf, np.inf, np.inf, np.inf, 8],
                     [np.inf, np.inf, np.inf, np.inf, np.inf, np.inf]],
                   index=['A', 'B', 'C', 'D', 'E', 'F'], columns=['A', 'B', 'C', 'D', 'E', 'F'])
Test_2

,A,B,C,D,E,F
A,inf,5.000000,4.000000,7.000000,6.000000,8.000000
B,inf,inf,7.000000,10.000000,9.000000,11.000000
C,inf,inf,inf,7.000000,6.000000,8.000000
D,inf,inf,inf,inf,5.000000,9.000000
E,inf,inf,inf,inf,inf,8.000000
F,inf,inf,inf,inf,inf,inf


In [91]:
F, T_s=UWPGMA(Test_2)
F

,F,DEBAC
F,inf,9.000000
DEBAC,inf,inf


In [92]:
T_s

'(F: 4.5, ((D: 2.5, E: 2.5): 4.0, (B: 3.0, (A: 2.0, C: 2.0): 3.0): 4.0): 4.5)'

In [93]:
F, T_s=UWPGMA(Test_2, weighted=False)
F

,F,DEBAC
F,inf,8.800000
DEBAC,inf,inf


In [94]:
T_s

'(F: 4.4, ((D: 2.5, E: 2.5): 3.75, (B: 3.0, (A: 2.0, C: 2.0): 3.0): 3.75): 4.4)'